In [ ]:
from importlib import reload
import ipynb.fs.full.notebook_viewer
reload(ipynb.fs.full.notebook_viewer)
from ipynb.fs.full.notebook_viewer import Viewer, root, bucket, fname2id, id2fname, split_dump
import pickle
from pprint import pprint
from dotdict import dotdict
from functools import reduce

from typing import *
import matplotlib.pyplot as plt
import random
import PIL
from pathlib import Path
import json
from collections import defaultdict
from vivid.utils import timer
import subprocess
from subprocess import PIPE
from tqdm.notebook import tqdm
from time import time
from functools import lru_cache
import numpy as np
from PIL import Image
import imagesize

In [ ]:
old = set(Path(root/"danbooru2019/512px.dat").read_text().strip().split())
safe =  set((root/"danbooru2020/512px.dat").read_text().strip().split())

readable_ori =  set((root/"danbooru2020/dump_readable_ori/0").read_text().strip().split())

fire_eggs = dict()
for k in "duplicates is_deleted no_humans not-image photo text_only_page".split():
    fire_eggs[k] = set((root/f"danbooru2020/Danbooru2019/reduce/{k}.txt").read_text().strip().split())
    
ge512 =  set((root/"danbooru2020/ge512.ids").read_text().strip().split())
ge1024 =  set((root/"danbooru2020/ge1024.ids").read_text().strip().split())

i = 2
if not "ids" in globals():
    with timer(prefix="read_id_pkl"): # 311[s]
        with open((root/f"danbooru2020/{i}_id.pkl"), "rb") as f:
            ids = pickle.load(f)
if not "tags" in globals():
    with timer(prefix="read_tag_pkl"): # 58[s]
        with open((root/f"danbooru2020/{i}_tag.pkl"), "rb") as f:
            tags = pickle.load(f)

In [ ]:
@lru_cache(maxsize=None)
def t(s):
    if "|" in s:
        retval = reduce(set.__or__, map(tags.get, s.split("|")))
    else:
        retval = reduce(set.__and__, map(tags.get, s.split("&")))
    return retval

@lru_cache(maxsize=None)
def load_json(fname):
    fname = str(fname)
    retval = dotdict({k.replace("detection_", "").replace("bbox_", "").replace("class_", ""): np.array(v) for k, v in json.loads(Path(fname).read_text()).items()})
    retval["fname"] = fname
    retval["id"] = fname2id(fname)
    return retval

def id2original(_id):
    return id2fname(_id, prefix="original", ext=lambda _id: ids[_id]["file_ext"])

def id2json(_id):
    return id2fname(_id, prefix="solo_json", bucket=lambda _: ".", ext=lambda _: "json")

def groupj(jfnames):
    r = dotdict()
    r.jfnames = tuple(map(str, jfnames))
    r.zero = set()
    r.one = set()
    r.multi = set()
    for fname in tqdm(r.jfnames):
        j = load_json(fname)
        r[j.id] = j
        w = np.where(j.label == 1)[0]
        if len(w) == 0:
            r.zero.add(j.id)
        elif len(w) == 1:
            r.one.add(j.id)
        else:
            r.multi.add(j.id)
    print(str(len(r.one)/len(r.jfnames)*100)[:2]+"%")
    return r

In [ ]:
solo = t("1girl|solo|1boy")-t("1girl&1boy")-t("2boys|2girls|3boys|3girls|4boys|4girls|5boys|5girls|6+boys|6+girls|multiple_boys|multiple_girls|multiple_views|variations")
white = t("white_background|transparent_background")
exc0 = t('|'.join([
    "no_humans|arachne|pokemon_(creature)|octoling|monster_girl|isabelle_(animal_crossing)|animal_nose|animal_hug",
    "robot|cyborg|mecha_musume|machinery|doll_joints|crewmate_(among_us)|kamen_rider_faiz|olimar",
    "red_skin|blue_skin|purple_skin|wrinkled_skin|green_skin|two-tone_skin",
    "monochrome|lineart|text_only_page|photo-referenced|3d|unconventional_media|comic",
    "full_armor|weapon|chibi|cannon|mask|eyepatch",
]))
exc1 = t('|'.join([
    "zoom_layer",
    "back|ass|from_behind|from_above|bent_over|dutch_angle|standing_split",
    "car|bicycle",
    "translation_request",
    "fox_tail",
    "otter_costume",
    "sonic_the_hedgehog",
    "mystery_skulls",
    "floating_head",
]))|reduce(set.__or__, fire_eggs.values())

fl  = [ solo&readable_ori        ] ; print("{}  {:>9,}  {}".format(len(fl)-1, len(fl[-1]), "solo"     ))
fl += [ fl[-1]-exc0              ] ; print("{}  {:>9,}  {}".format(len(fl)-1, len(fl[-1]), "exc0"     ))
fl += [ fl[-1]&tags["standing"]  ] ; print("{}  {:>9,}  {}".format(len(fl)-1, len(fl[-1]), "standing" ))
fl += [ fl[-1]&ge512             ] ; print("{}  {:>9,}  {}".format(len(fl)-1, len(fl[-1]), "ge512"    ))
fl += [ fl[-1]&ge1024            ] ; print("{}  {:>9,}  {}".format(len(fl)-1, len(fl[-1]), "ge1024"   ))
#fl += [ fl[-1]&tags["highres"]   ] ; print("{}  {:>9,}  {}".format(len(fl)-1, len(fl[-1]), "highres"  ))
fl += [ fl[-1]&white             ] ; print("{}  {:>9,}  {}".format(len(fl)-1, len(fl[-1]), "white"    ))
fl += [ fl[-1]-tags["censored"]  ] ; print("{}  {:>9,}  {}".format(len(fl)-1, len(fl[-1]), "censored" ))
fl += [ fl[-1]&safe              ] ; print("{}  {:>9,}  {}".format(len(fl)-1, len(fl[-1]), "safe"     ))
fl += [ fl[-1]&tags["full_body"] ] ; print("{}  {:>9,}  {}".format(len(fl)-1, len(fl[-1]), "full_body"))
nofull = fl[7]-exc1                ; print("{}  {:>9,}  {}".format("x", len(nofull), "nofull"))
nofullj = groupj(map(id2json, nofull))
v3 = nofull&nofullj.one            ;  print("{}  {:>9,}  {}".format("x", len(v3), "v3"))

In [ ]:
v3_score = list(v3)
v3_score.sort(key = lambda _id: int(ids[_id]["score"]), reverse=True)
Viewer(map(id2original, v3_score), seed=-1, name="v3_score", N=11)

In [ ]:
def convert_option(_id, dname, xc=0.5, yc=0.28, yw=0.235, size=512):
    fname = id2original(_id)
    j = load_json(id2json(_id))
    src = dotdict()
    dst = dotdict()
    src.X, src.Y = imagesize.get(fname)
    i = np.where(j.label==1)[0][0]
    src.Xmin = src.X*j.xmin[i]
    src.Xmax = src.X*j.xmax[i]
    src.Ymin = src.Y*j.ymin[i]
    src.Ymax = src.Y*j.ymax[i]
    src.Ywid = src.Ymax-src.Ymin
    src.Xcen = (src.Xmax+src.Xmin)/2
    src.Ycen = (src.Ymax+src.Ymin)/2
    dst = dotdict()
    dst.Xcen = size*xc
    dst.Ycen = size*yc
    dst.Ywid = size*yw
    z = dst.Ywid/src.Ywid
    option = [
        "-background", "white",
        "-resize", f"{round(100*z)}%",
        "-crop", f"{size}x{size}+{round(z*src.Xcen-dst.Xcen)}+{round(z*src.Ycen-dst.Ycen)}!",
        "-flatten",
        "-strip",
        "-quality", "100",
        f"{fname}",
        f"PNG24:{dname}",
    ]
    return option

In [ ]:
options = list()
for _id in tqdm(v3_score):
    option = " ".join(convert_option(_id, id2fname(_id, prefix="v3")))
    options.append(option)

In [ ]:
split_dump(options, "option_v3")

In [ ]:
sample = list()
for fname in tqdm(list((root/"danbooru2020/v3").glob("**/*.png"))):
    sample.append( np.array(Image.open(fname)) )
Image.fromarray(np.mean(sample, axis=0).astype("uint8"))